In [4]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd 
import re 

# To close the web browser 
# from selenium import webdriver   # for webdriver
# from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
# from selenium.webdriver.chrome.options import Options  # for suppressing the browser

## create dataframe

In [95]:
top100_whisky = pd.read_csv('Top100_whisky.csv')

# create whisky details csv for hkls 
whisky_name = top100_whisky.iloc[:,0:2].copy()
wastons_name = top100_whisky.iloc[:,6:8].copy()
watsons_details = pd.concat([whisky_name, wastons_name], axis = 1)
watsons_details.head()

# add new columns in hkls_details
watsons_details['Price'] = pd.Series(dtype='str')
watsons_details['Stock'] = pd.Series(dtype='str')
watsons_details['Volume'] = pd.Series(dtype='str')

# obtain links 
page = watsons_details['watsons_link']

# Web scrape price, volume and stock
for index, p in enumerate(page):
    if len(str(p))>5:
    # Oillio's code to stop the banner
        capa = DesiredCapabilities.CHROME
        capa["pageLoadStrategy"] = "none"
        driver = webdriver.Chrome('../chromedriver',desired_capabilities=capa)
        driver.get(p)
        time.sleep(5)
        driver.execute_script("window.stop();")
        driver.maximize_window()    

        try:
            watsons_details['Price'][index] = re.search(r'(\$)(\d+,\d+)|(\$)(\d+)', driver.find_element_by_class_name("offer-price").find_element_by_class_name("price").text).group(2)
        except:
            watsons_details['Price'][index]= re.search(r'(\$)(\d+,\d+)|(\$)(\d+)',driver.find_element_by_class_name("regular-price").text).group(2)
                    
        try: 
            if driver.find_element_by_class_name("addProduct").text == 'Out of Stock':
                watsons_details['Stock'][index] = False
            else:
                watsons_details['Stock'][index] = True
        except:
            watsons_details['Stock'][index] = True
        try:
            watsons_details['Volume'][index] = str(int(re.search(r'(\d+)(CL)$', driver.find_element_by_xpath('//*[@id="overview"]/div/div/div[2]/table/tbody/tr[1]/td[2]').text).group(1))*10)+'ml'
        except:
            watsons_details['Volume'][index] = 'Not Available'

watsons_details.to_csv('watsons_details.csv', index=False)

KeyboardInterrupt: 

## Update dataframe 

In [118]:
def update_watsons_detail():
    watsons_details = pd.read_csv('watsons_details.csv')
    page = watsons_details['watsons_link']

    for index, p in enumerate(page):
        if len(str(p))>5:
        # Oillio's code to stop the banner
            capa = DesiredCapabilities.CHROME
            capa["pageLoadStrategy"] = "none"
            driver = webdriver.Chrome('../chromedriver',desired_capabilities=capa)
            driver.get(p)
            time.sleep(7)
            driver.execute_script("window.stop();")
            driver.maximize_window()    

            try:
                watsons_details['Price'][index]= int(re.search(r'\$\d+,\d+|\$\d+',driver.find_element_by_class_name("regular-price").text).group().replace(',', '').replace('$', ''))
            except:
                watsons_details['Price'][index] = int(re.search(r'\$\d+,\d+|\$\d+', driver.find_element_by_class_name("offer-price").find_element_by_class_name("price").text).group().replace(',', '').replace('$', ''))
            try: 
                if driver.find_element_by_class_name("addProduct").text == 'Out of Stock':
                    watsons_details['Stock'][index] = False
                else:
                    watsons_details['Stock'][index] = True
            except:
                watsons_details['Stock'][index] = True
      
    watsons_details.to_csv('watsons_details.csv', index=False)
    return watsons_details

update_watsons_detail()
    

,Name,Year,watsons_name,watsons_link,Price,Stock,Volume
0,Lagavulin,16,Lagavulin 16 Years Old Single Malt Whisky,https://www.watsonswine.com/en/wine/lagavulin-...,910,True,700ml
1,Ardbeg,10,Ardbeg 10 Year Old Islay Single Malt Whisky,https://www.watsonswine.com/en/wine/ardbeg-10-...,680,True,700ml
2,Balvenie,12,Balvenie Doublewood 12 Years Single Malt Scotc...,https://www.watsonswine.com/en/wine/balvenie-d...,598,True,700ml
3,Laphroaig,10,Laphroaig 10 Year Old Single Malt Whisky,https://www.watsonswine.com/en/wine/laphroaig-...,728,False,700ml
4,Glenmorangie,10,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
95,Chivas Regal,18,Chivas Regal 18 Years Scotch Whisky,https://www.watsonswine.com/en/wine/chivas-reg...,698,True,700ml
96,Bruichladdich,the classic laddie,Bruichladdich Classic Laddie Scottish Barley,https://www.watsonswine.com/en/wine/bruichladd...,668,True,700ml
97,Glen Moray,15,NaN,NaN,NaN,NaN,NaN
98,Aberfeldy,16,NaN,NaN,NaN,NaN,NaN


In [119]:
watsons_details=pd.read_csv('watsons_details.csv')
watsons_details.head()

,Name,Year,watsons_name,watsons_link,Price,Stock,Volume
0,Lagavulin,16,Lagavulin 16 Years Old Single Malt Whisky,https://www.watsonswine.com/en/wine/lagavulin-...,910.0,True,700ml
1,Ardbeg,10,Ardbeg 10 Year Old Islay Single Malt Whisky,https://www.watsonswine.com/en/wine/ardbeg-10-...,680.0,True,700ml
2,Balvenie,12,Balvenie Doublewood 12 Years Single Malt Scotc...,https://www.watsonswine.com/en/wine/balvenie-d...,598.0,True,700ml
3,Laphroaig,10,Laphroaig 10 Year Old Single Malt Whisky,https://www.watsonswine.com/en/wine/laphroaig-...,728.0,False,700ml
4,Glenmorangie,10,NaN,NaN,NaN,NaN,NaN


## Select Whisky  

In [7]:
def watsons_whisky(index):
    watsons_details = pd.read_csv('watsons_details.csv')
    name, address, price = None, None, None

    if watsons_details['Stock'][index] == True:
        name = 'Watson\'s wine  (23 Stores with online shop)'
        address = 'https://www.watsonswine.com/en'
        price = watsons_details['Price'][index]

    return [name,address,price]

In [8]:
# watsons_whisky['Stock'][2] ---> with stock
watsons_whisky(2)

["Watson's wine  (23 Stores with online shop)",
 'https://www.watsonswine.com/en',
 598.0]

In [11]:
# watsons_whisky['Stock'][3] ---> no stock
watsons_whisky(3)

[None, None, None]

In [10]:
# watsons_whisky['Stock'][4] ---> not selling
watsons_whisky(4)

[None, None, None]

In [65]:
# Scraping by content page but number of listing whisky not match 
page =['https://www.watsonswine.com/en/wine-list/whiskies/c/10010211','https://www.watsonswine.com/en/wine-list/whiskies/c/10010211?q=%3Adiscount&page=1&resultsForPage=72&text=&sort=']

name, price, stock, volume = [],[],[],[]

for p in page:
    capa = DesiredCapabilities.CHROME
    capa["pageLoadStrategy"] = "none"
    driver = webdriver.Chrome('../chromedriver',desired_capabilities=capa)
    driver.get(p)
    time.sleep(5)
    driver.execute_script("window.stop();")
    driver.maximize_window()    

    whiskies = driver.find_elements_by_class_name('intro')
    whiskies_ava = driver.find_elements_by_class_name('addProduct')

    for i in range(len(whiskies)):
        name.append(whiskies[i].find_element_by_class_name('productName').text)
        volume.append(whiskies[i].find_element_by_class_name('size').text)
        try:
            price.append(whiskies[i].find_element_by_class_name('s1').text)
        except:
            price.append(whiskies[i].find_element_by_class_name('s1 colorBlack').text)
        try:
            if whiskies_ava[i].find_element_by_class_name('out-stork'):
                stock.append(False)
        except: 
            stock.append(True)

for index, list_w in enumerate(watsons_details['watsons_name']):
    for i, name_w in enumerate(name):
        if str(list_w).lower() == str(name_w).lower():
            watsons_details['Price'][index] = re.search(r'\$\d+,\d+|\$\d+',price[i]).group()
            try:
                watsons_details['Volume'][index] = str(int(re.search(r'(\d+)(CL)$', volume[i]).group(1))*10)+'ml'
            except:
                watsons_details['Volume'][index] = 'Not Available'
            watsons_details['Stock'][index] = stock[i]

watsons_details.to_csv('watsons_details.csv', index=False)

print (len(name))

71
